# Automate the Cleaning Process

In [1]:
import os
import json
import pandas as pd
import numpy as np
import re

# Dependencies required for the uploading process
# import time
# from sqlalchemy import create_engine
# import psycopg2
# from config import db_password




# Functions created during the initial cleaning
- Each one takes a string as input:
 - cleaning_movie
     - reduces the sheer variety of keys the movie object has in wiki_movie
 - find_dollars
     - finds the dollar values hidden amoung the strings and lists
 - parse_dollars
     - converts the dollar values found by find_dollars into numeric values
 - find_dates
     - finds dates hidden in strings and lists to be converted to datetime objects with pd.to_datetime
 - find_time
     - finds the movie duration in minutes from amoung the strings and lists and converts them into numeric values

### cleaning_movie

In [2]:
def cleaning_movie(movie):
    """wiki_movie_file is a json file containing a list of dictionaries with each index as a different movie 
    with different key:value pairs. This function is specially designed to clean that specific dataset. 
    Wikipedia is managed by many people and they do not all use the same words.
    """
    alternate_titles = dict()
    fixed_movie = dict(movie)
###  Start with alternate titles stored in language keys and merge them into one.
###  Language keys i could find:      
    language_keys = ['Also known as','Arabic','Cantonese','Chinese','French', 'Hangul','Hebrew','Hepburn','Japanese','Literally',
        'Mandarin','McCune–Reischauer','Original title','Polish', 'Revised Romanization','Romanized','Russian',
        'Simplified','Traditional','Yiddish']
    for key in language_keys:
        if key in fixed_movie.keys():
            alternate_titles[key] = fixed_movie[key]
            fixed_movie.pop(key)
        else:
            pass
##  if there were alternate titles, add them to the movie.       
    if len(alternate_titles) > 0:
        fixed_movie['alternate_titles'] = alternate_titles
    else:
        pass
        
###  Alternative titles are fixed, now merge columns that are similar.     
    keys_to_merge = {'Director':'Directed by', 'Country': 'Country of origin', 'Distributor(s)':'Distributed by',
                     'Editor(s)':'Edited by',  'Language':'Original language(s)', 'Producer(s)':'Produced by',
                     'Genre(s)': 'Genre', 'Composer(s)': ['Music by', 'Theme music composer'], 
                     'Release date': ['Release(s)', 'Original release'], 'Distributor(s)':['Distributed by','Distributor'],
                     'Writer(s)':['Written by', 'Story by', 'Screenplay by', 'Screen story by', 'Adaptation by'],
                     'Production Comapany': ['Production company', 'Production company(s)', 'Productioncompanies ','Productioncompany ']  }
##  item = key, from the key:value pair and the key I want; values = the movie key(s) I do not want. 
    for item in keys_to_merge: 
        if type(keys_to_merge[item]) == type(list()):
            for n in keys_to_merge[item]:
                if n in fixed_movie.keys():
                    fixed_movie[item] = fixed_movie.pop(n)
                else:
                    pass
        else: 
            if keys_to_merge[item] in fixed_movie.keys():
                fixed_movie[item] = fixed_movie.pop(keys_to_merge[item])
            else:
                pass
    
    return fixed_movie



### find_dollars

In [48]:
def find_dollars(s):
    # declared patterns
    # searching for pattern that is similar to: "$45.3 million/billion"
    p1 = r'(\$\s?\d+\.?\d*\s*[bm]illi?on)'      
    # searching for pattern that is similar to $123,456,789.0
    p2 = r'(\$\s?\d+(?:[,\.]\d{3})+\.?\d*)(?!\s*[bm]illi?on)' 
    # modified pattern #1; includes a range $43.5-45.7 Million
    p3 = r"(\$\d*\.?\d*)(?:[-—–]\d*\.?\d*)(\s[bm]illi?on)" # modified pattern #1 version 3
    # searcing for $100 to $999999 (no commas)
    p4 = r"(\$\d{3,6})" 
    if len(re.findall(p1, str(s), flags=re.IGNORECASE)) == 1:
        num = re.findall(p1, str(s), flags=re.IGNORECASE)[0]
        return num
    elif len(re.findall(p2, str(s), flags=re.IGNORECASE)) == 1:
        num = re.findall(p2, str(s), flags=re.IGNORECASE)[0]
        return num
    elif len(re.findall(p1, str(s), flags=re.IGNORECASE)) > 1: 
        num = re.findall(p1, str(s), flags=re.IGNORECASE)[1]
        return num
    elif len(re.findall(p2, str(s), flags=re.IGNORECASE)) > 1:  
        num = re.findall(p2, str(s), flags=re.IGNORECASE)[1]
        return num
    elif len(re.findall(p3, str(s), flags=re.IGNORECASE)) > 0:
        fix = re.findall(p3, str(s), flags=re.IGNORECASE)
        num = fix[0][0] + fix[0][1]
        return num
    elif len(re.findall(p4, str(s), flags=re.IGNORECASE)) > 0:
        num = re.findall(p4, str(s), flags=re.IGNORECASE)[0].replace(" ", "")
        return num
    else:
        return np.nan

### parse_dollars

In [49]:
# Created during initial cleaning for budget and revenue/box office
# parse the dollar values found with find_dollars() and durn them into numeric values
def parse_dollars(s):   
    # patterns to change into numbers
    p1 = "(\$\d+\.?\d*\s*milli?on)" # pulled from pattern #1 above
    p2 = "(\$\d+\.?\d*\s*billi?on)" # pulled from pattern #1 above
    p3 = "(\$\d+(?:[,\.]\d{3})+\.?\d*)"  # pulled from pattern #2 above
    
    # if s is string, then change it, otherwise np.nan
    if type(s) == str:
  
        # if input is of the form $###.## Million, pattern #1
        if re.match(p1, s, flags=re.IGNORECASE):
            # Remove dollar sign and "million", convert to float and multiply by 1,000,000
            num = float(re.search("(\d+\.?\d*)", s)[0])*1000000
            
            # return value
            return num
            
        # elif input is of the form $###.## Billion, formerly pattern #1 also, now pattern #2
        elif re.match(p2, s, flags=re.IGNORECASE):
        # Remove dollar sign and "billion", convert to float and multiply by 1,000,000,000
            num = float(re.search("(\d+\.?\d*)", s)[0])*1000000000
            # return value
            return num

            
        # elif input is of the form $###,###,###, formerly pattern #2, now pattern #3
        elif re.match(p3, s, flags=re.IGNORECASE):
            # Remove dollar sign and commas, convert to float
            num = float(re.findall(r"(?:\$)(\d+(?:[,\.]\d{3})+\.?\d*)", s)[0].replace(",", ""))

            # return value
            return num
        else: 
            if len(s) <= 6:
                if re.match("(\$\d{1,6}$)", s, flags=re.IGNORECASE):
                    num = float(s.replace("$", ""))
                    return num

    else: 
        return np.nan


### find_dates

In [64]:
def find_dates(s):
    # declared patterns
    p1 = r"(?:January|February|March|April|May|June|July|August|September|October|November|December)\s*[123]?\d,\s*\d{4}"
    p2 = r"[123]?\d\s*(?:January|February|March|April|May|June|July|August|September|October|November|December)\s*\d{4}"
    p3 = r"(?:January|February|March|April|May|June|July|August|September|October|November|December),?\s*\d{4}"
    p4 = r"(\d{4})(?!\S)"
    p5 = r"(\d{4}-[01]?\d-[0123]?\d)"
    if re.findall(p1, str(s), flags=re.IGNORECASE):  # Month DD, YYYY
        spam = re.findall(p1, str(s), flags=re.IGNORECASE)[0]
        return spam
    elif re.findall(p2, str(s), flags=re.IGNORECASE): # DD Month YYYY
        spam = re.findall(p2, str(s), flags=re.IGNORECASE)[0]
        return spam
    elif re.findall(p3, str(s), flags=re.IGNORECASE): # Month YYYY
        spam = re.findall(p3, str(s), flags=re.IGNORECASE)[0]
        return spam
    elif re.findall(p5, str(s), flags=re.IGNORECASE): # YYYY-MM-DD
        spam = re.findall(p5, str(s), flags=re.IGNORECASE)[0]
        return spam
    elif re.findall(p4, str(s), flags=re.IGNORECASE): # YYYY
        spam = re.findall(p4, str(s), flags=re.IGNORECASE)[0]
        return spam
    else:
        return np.nan



### find_time

In [65]:
def find_time(s):
    # Declared patterns
    p1 = r"(\d+)(?:', ')?\s*min\.?(?:utes)?" # 120 minutes, 120 min, 120 min., '120', 'min'
    p2 = r"((?:\d+\s*h)?\s*\d+m)"# 1h 48m, 70m
    p3 = r"(\d+\s*hours?)" # one record says "4 Hours"
    if type(s) == float:
        return s
    if re.findall(p1, str(s), flags=re.IGNORECASE): # 120 minutes, 120 min, 120 min., '120', 'min'
        spam = int(re.findall(p1, str(s), flags=re.IGNORECASE)[0])
        return spam
    elif re.findall(p2, str(s), flags=re.IGNORECASE): # 1h 48m, 70m
        spam = re.findall(p2, str(s), flags=re.IGNORECASE)[0]
        egg = int(re.findall(r"(\d+)m$", spam, flags=re.IGNORECASE)[0])
        try:
            hours = int(re.findall(r"(\d+)h", spam, flags=re.IGNORECASE)[0])*60 
            egg+=hours
        except:
            pass

        return egg
    elif re.findall(p3, str(s), flags=re.IGNORECASE): # 1 record says "4 hours" 
        spam = re.findall(p3, str(s), flags=re.IGNORECASE)[0]
        egg = int(re.findall(r"(\d+)\s*h", spam, flags=re.IGNORECASE)[0])*60
        return egg
    else:
        return float(np.nan)

# Automate the Process

In [1]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load(wiki_json, kaggle_csv, ratings_csv):
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_df = pd.read_csv(kaggle_csv, sep=",", header=0, low_memory=False)
    ratings_df = pd.read_csv(ratings_csv, sep=",", header=0)

    # 3. Open the read the Wikipedia data JSON file.
    with open(wiki_json, 'r') as file:
        wiki = json.load(file)
        file.close()
    
    
    # 4. Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki if (('imdb_link' and ('Directed by' or 'Director')) in movie.keys()) and ('No. of episodes' not in movie.keys())]
    # 5. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie. Read in the cleaned movies list as a DataFrame.
    
    cleaned_movies = [cleaning_movie(movie) for movie in wiki_movies]
    wiki_df = pd.DataFrame(cleaned_movies)
    
    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    imdb_pattern = r"(tt\d{7})"
    try:
        wiki_df['imdb_id'] = wiki_df['imdb_link'].str.extract(imdb_pattern)
    except TypeError: 
        print("imdb_pattern: TypeError, data cell type not string")
    except KeyError: 
        print("'imdb_id' or 'imdb_link' keys not available in wiki_movies")
    except: 
        print("Differnt Error")
    
    wiki_df = pd.DataFrame(wiki_df.drop_duplicates(subset='imdb_id'))
        
    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    columns_to_keep = [column for column in wiki_df if wiki_df[column].isnull().sum()< len(wiki_df)*0.9]
    wiki_df = wiki_df[columns_to_keep]

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_df['Box office'].dropna()

    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ''.join(x) if type(x) != str else x)

    # 10. Clean the box office column in the wiki_movies_df DataFrame with the parse_dollars function.
    wiki_df['box_office'] =  box_office.apply(lambda x: parse_dollars(find_dollars(x)))
    
    # 11. Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_df['Budget'].dropna()
    budget = budget.apply(lambda x: ''.join(x) if type(x) != str else x)
    wiki_df['budget'] = budget.apply(lambda x : parse_dollars(find_dollars(x)))

    # 12. Clean the release date column in the wiki_movies_df DataFrame.
    release_date = wiki_df['Release date'].dropna()
    wiki_df['release_date'] = pd.to_datetime(release_date.apply(lambda x : find_dates(x)), infer_datetime_format=True)

    # 13. Clean the running time column in the wiki_movies_df DataFrame.
    runtime = wiki_df['Running time'].dropna()
    wiki_df['running_time'] = runtime.apply(lambda x:  find_time(x))

        
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_df, kaggle_df, ratings_df

In [122]:
# 17. Create the path to your file directory and variables for the three files.

# The Wikipedia data
wiki_file = os.path.join('Data', 'wikipedia-movies.json')
# The Kaggle metadata
kaggle_file = os.path.join('Data', 'movies_metadata.csv')
# The MovieLens rating data.
ratings_file = os.path.join('Data', 'ratings.csv')



In [123]:
wiki_movies_df, kaggle_metadata, ratings = extract_transform_load(wiki_file, kaggle_file, ratings_file)

C:\Users\Nick\AppData\Local\Temp\ipykernel_12816\333603968.py:1: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  wiki_movies_df, kaggle_metadata, ratings = extract_transform_load(wiki_file, kaggle_file, ratings_file)


In [124]:
# 18. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Running time,Country,...,Editor(s),Producer(s),Composer(s),Writer(s),Production Comapany,imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,United States,...,Michael Tronick,"[Steve Perry, Joel Silver]","[Cliff Eidelman, Yello]","[David Arnott, James Cappe, Daniel Waters]",Silver Pictures,tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,United States,...,Howard E. Smith,"[Ric Kidney, Robert Redlin]",Maurice Jarre,"[James Foley, Robert Redlin]",Avenue Pictures,tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,United States,...,"[John Bloom, Lois Freeman-Fox]",Daniel Melnick,Charles Gross,"[John Eskow, Richard Rush]","[Carolco Pictures, IndieProd Company]",tt0099005,57718089.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,United States,...,Susan E. Morse,Robert Greenhut,NaN,Woody Allen,NaN,tt0099012,7331647.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",95 minutes,US,...,David Stiven,John Cornell,Maurice Jarre,Paul Hogan,NaN,tt0099018,6939946.0,25000000.0,1990-12-19,95.0


In [125]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Director',
 'Distributor(s)',
 'Editor(s)',
 'Producer(s)',
 'Composer(s)',
 'Writer(s)',
 'Production Comapany',
 'imdb_id',
 'box_office',
 'budget',
 'release_date',
 'running_time']

In [126]:
wiki_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7098
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   url                  7032 non-null   object        
 1   year                 7032 non-null   int64         
 2   imdb_link            7031 non-null   object        
 3   title                7031 non-null   object        
 4   Based on             2181 non-null   object        
 5   Starring             6850 non-null   object        
 6   Cinematography       6343 non-null   object        
 7   Release date         7000 non-null   object        
 8   Running time         6893 non-null   object        
 9   Country              6798 non-null   object        
 10  Language             6948 non-null   object        
 11  Budget               4738 non-null   object        
 12  Box office           5486 non-null   object        
 13  Director             7032 non-nul